In [1]:
import os

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(
    openai_api_key = os.getenv('OPENAI_API_KEY'),
    temperature=0,
    model='gpt-3.5-turbo-instruct'
)

In [4]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description="Useful when you need to answer questions about math."
)

tools = [math_tool]

In [5]:
tools[0].name, tools[0].description

In [6]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [16]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [17]:
zero_shot_agent("What is (3.311 + 8.04)^ 2.4?")

In [18]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# 
# prompt = PromptTemplate(
#     input_variables=["query"],
#     template="{query}"
# )
# 
# llm_chain = LLMChain(llm=llm, prompt=prompt)
# 
# llm_tool = Tool(
#     name="Language Model",
#     func=llm_chain.run,
#     description="Use this tool for general purpose queries and logic."
# )

In [21]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables.base import RunnableSequence
from langchain.agents import Tool

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_sequence = prompt | llm

llm_tool = Tool(
    name="Language Model",
    func=llm_sequence.invoke,
    description="Use this tool for general purpose queries and logic."
)

In [22]:
tools.append(llm_tool)

In [23]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3    
)

In [24]:
zero_shot_agent("Tell me about Gettysberg Speech.")

In [25]:
zero_shot_agent("What is (21 + 67)^2?")

In [26]:
openai_api_key = os.getenv('OPENAI_API_KEY')

from langchain_openai import OpenAI

llm = OpenAI(
    openai_api_key = openai_api_key,
    temperature=0,
)

In [27]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
        
    return result

In [28]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [29]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False)
)

In [30]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [31]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)]
]

In [32]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )
    
    with engine.begin() as conn:
        conn.execute(stmt)   

In [33]:
for obs in observations:
    insert_obs(obs)

In [39]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

In [40]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

In [41]:
result = count_tokens(
    agent_executor,
    "What is the product of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' on January 3rd and the ratio " +
    "between the same stock prices on January 4th?"    
)

In [47]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [48]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [53]:
from langchain.agents import initialize_agent

conversation_agent = initialize_agent(
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [54]:
result = count_tokens(
    conversation_agent,
    "What's the result of an investment of $10000 growing at 8% annually for 5 years with compound interest?"
)

In [52]:
math_tool = Tool(name='Calculator',func=llm_math.run, description='Useful when you need to answer questions about math.')
tools = [math_tool]

In [55]:
result = count_tokens(
    conversation_agent,
    "if we start with $15000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared tp the previous scenario?"
)

In [56]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore = DocstoreExplorer(Wikipedia())    
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="search wikipedia"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [57]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [59]:
count_tokens(docstore_agent, "What is the significance of Olympic games?")

In [61]:
from langchain_openai import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERP_API_KEY"))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="Google search"
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [62]:
self_ask_with_search